# Description


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# 1. Prepare data

## 1.1 Load basic data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
df = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
df.index = df.index.map(str)
feats = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
path_save = f"{path}/special/053_proof_that_immunodata_is_shit"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

## 1.2 Get data with nans

In [ ]:
files = [
    "Aging L, Q, H, I",
    "Aging-Covid_05.01.2022",
    "Aging-Covid-05.05.22",
    "Covid_results_02_2021",
    "Covid-25.11.20",
    "MULTIPLEX_20_11_2020_ AGING",
    "Yakutiya + TR",
    "Мультиплекс_Agind&Covid",
]
df_immuno_genes = pd.read_excel(f"{path}/data/immuno/immuno_markers_genes.xlsx")
dict_immuno_genes = dict(zip(df_immuno_genes['immuno_marker'], df_immuno_genes['gene']))

dfs_files = []
for file in files:
    df_file = pd.read_excel(f"{path}/data/immuno/files/processed/{file}.xlsx", index_col="Sample")
    df_file.rename(columns=dict_immuno_genes, inplace=True)
    df_file = df_file.loc[:, feats]

    # duplicates processing
    if file == "MULTIPLEX_20_11_2020_ AGING":
        df_file_doubled_unique = df_file.loc[~df_file.index.duplicated(keep=False), :]
        df_file_doubled_1 = df_file.loc[df_file.index.duplicated(keep='first'), :]
        df_file_doubled_2 = df_file.loc[df_file.index.duplicated(keep='last'), :]
        df_file_duplicates_final = pd.concat([df_file_doubled_2, df_file_doubled_unique], axis=0)
        df_file = df_file_duplicates_final
    df_file_duplicates = df_file.loc[df_file.index.duplicated(keep=False), :]
    if df_file_duplicates.shape[0] > 0:
        print(df_file_duplicates.index)

    df_file.replace(r'^([<>].*)$', 'NaN', inplace=True, regex=True)
    df_file = df_file.apply(pd.to_numeric, errors='coerce')
    dfs_files.append(df_file)

df_w_nans = pd.concat(dfs_files, verify_integrity=False)
df_w_nans.index = df_w_nans.index.map(str)
df_w_nans = df_w_nans.loc[df.index.values, :]
df_w_nans.to_excel(f"{path_save}/df_w_nans.xlsx", index_label="Index")

# Checking values
df_diff = df.loc[df.index.values, feats] - df_w_nans.loc[df.index.values, feats]
df_diff = df_diff.fillna(0.0)
max_diff = df_diff.values.max()
print(f"max_diff: {max_diff}")

## 1.3 Problem definitions

In [ ]:
subsets = {
    'All Samples': {
        'color': 'black',
        'samples': df.index.values,
        'path': 'All'
    },
    "File Aging L, Q, H, I": {
        'color': px.colors.qualitative.Plotly[0],
        'samples': df.index[df['file'] == "Aging L, Q, H, I"].values,
        'path': 'Files/Aging L, Q, H, I'
    },
    "File Aging-Covid_05.01.2022": {
        'color': px.colors.qualitative.Plotly[1],
        'samples': df.index[df['file'] == "Aging-Covid_05.01.2022"].values,
        'path': 'Files/Aging-Covid_05.01.2022'
    },
    "File Aging-Covid-05.05.22": {
        'color': px.colors.qualitative.Plotly[2],
        'samples': df.index[df['file'] == "Aging-Covid-05.05.22"].values,
        'path': 'Files/Aging-Covid-05.05.22'
    },
    "File Covid_results_02_2021": {
        'color': px.colors.qualitative.Plotly[3],
        'samples': df.index[df['file'] == "Covid_results_02_2021"].values,
        'path': 'Files/Covid_results_02_2021'
    },
    "File Covid-25.11.20": {
        'color': px.colors.qualitative.Plotly[4],
        'samples': df.index[df['file'] == "Covid-25.11.20"].values,
        'path': 'Files/Covid-25.11.20'
    },
    "File MULTIPLEX_20_11_2020_ AGING": {
        'color': px.colors.qualitative.Plotly[5],
        'samples': df.index[df['file'] == "MULTIPLEX_20_11_2020_ AGING"].values,
        'path': 'Files/MULTIPLEX_20_11_2020_ AGING'
    },
    "File Yakutiya + TR": {
        'color': px.colors.qualitative.Plotly[6],
        'samples': df.index[df['file'] == "Yakutiya + TR"].values,
        'path': 'Files/Yakutiya + TR'
    },
    "File Мультиплекс_Agind&Covid": {
        'color': px.colors.qualitative.Plotly[7],
        'samples': df.index[df['file'] == "Мультиплекс_Agind&Covid"].values,
        'path': 'Files/Мультиплекс_Agind&Covid'
    },
    "Controls": {
        'color': 'lawngreen',
        'samples': df.index[(df['Status'] == "Control") | (df['COVID-19 stage'] == "Reconvalescent") ].values,
        'path': 'Status/Controls'
    },
    "СOVID-19 Acute and Dynamics": {
        'color': 'crimson',
        'samples': df.index[df['COVID-19 stage'].isin(['Acute', 'Dynamics'])].values,
        'path': 'Status/COVID-19'
    },
    "Down Syndrome": {
        'color': 'darkorchid',
        'samples': df.index[df['Down syndrome status'].isin(['Down Syndrome'])].values,
        'path': 'Status/DownSyndrome'
    },
    "ESRD": {
        'color': 'saddlebrown',
        'samples': df.index[df['Status'].isin(['ESRD'])].values,
        'path': 'Status/ESRD'
    },
}

# 2. NaN analysis

In [ ]:
for subset_name, subset in subsets.items():

    path_curr = f"{path_save}/02_nan_analysis/{subset['path']}"
    pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

    df_nan = df_w_nans.loc[subset['samples'], feats].isna().sum(axis=0).to_frame(name="Number of NaNs")
    df_nan["% of NaNs"] = df_nan["Number of NaNs"] / df.shape[0] * 100
    df_nan["Number of not-NaNs"] = df_w_nans.loc[subset['samples'], feats].notna().sum(axis=0)
    df_nan['Color'] = 'white'
    df_nan.loc[df_nan["% of NaNs"] < 10, 'Color'] = subset['color']
    df_nan.sort_values(["% of NaNs"], ascending=[False], inplace=True)
    df_nan.to_excel(f"{path_curr}/df_nan.xlsx", index_label="Features")

    plt.figure(figsize=(14, 4))
    plt.xticks(rotation=90)
    sns.set_theme(style='whitegrid')
    barplot = sns.barplot(
        data=df_nan,
        x=df_nan.index,
        y=f"% of NaNs",
        edgecolor='black',
        palette=df_nan['Color'].values,
        dodge=False
    )
    barplot.set_title(subset_name)
    plt.savefig(f"{path_curr}/barplot.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_curr}/barplot.pdf", bbox_inches='tight')
    plt.close()